 ## Попытка реализации алгоритма Литтла "Метод ветвей и границ" для решения задачи коммивояжера

### Использованная литература:
1. https://uchimatchast.ru/teoriya/algoritm-littla-primer-1/
2. http://galyautdinov.ru/post/zadacha-kommivoyazhera
3. http://www.math.nsc.ru/LBRT/k4/or/or_part4.pdf
4. https://math.semestr.ru/kom/index.php


In [253]:
import numpy as np

In [254]:
def distance_matrix(dict_coord, inf=inf): # В нашем случае симметричная матрица
    """
    Функция делает из координатной сетки dict_coord матрицу расстояний matrix (тип numpy.ndarray),
    добавляя числовые индексы слева и сверху по строкам и столбцам соответственно.
    
    dict_coord: словарь с координатами узлов
    inf: обозначает поля движения города в самого себя, а также поле пересечения индексов matrix[0][0]
    return: matrix - матрица расстояний
    """
    matrix = []
    for i in dict_coord.values():
        for j in dict_coord.values():
            if i!=j:
                matrix.append(((i[0] - j[0]) ** 2 + (i[1] - j[1]) ** 2) ** 0.5)
            else:
                matrix.append(inf)
    matrix =np.array(matrix).reshape(len(dict_coord), len(dict_coord))
    indx_from_city = np.array(list((dict_coord.keys()))).reshape(len(dict_coord), 1)
    matrix = np.append(indx_from_city, matrix, axis=1)
    indx_in_city = np.concatenate((np.array([inf]), np.array(list((dict_coord.keys())))), axis=0)
    matrix = np.append([indx_in_city], matrix, axis=0)
    return matrix


In [255]:
def reduce_matrix(reduce_m, H):
    """
    Функция делает редукцию матрицы reduce_m и вычисляет нижнюю границу H
    
    reduce_m: матрица которую редуцируем
    H: нижняя граница до редуцирования
    return: reduce_m - рецуцированная матрица
            H - нижняя граница после редуцирования
    """
    min_d_i = reduce_m[1:, 1:].min(axis=1) #минимумы по строкам
    for cnt, el in enumerate(reduce_m.T): #вычитание минимумов по строкам
        if not cnt:
            continue
        el[1:]-= min_d_i

    min_d_j = reduce_m[1:, 1:].min(axis=0) #минимумы по столбцам
    for cnt, el in enumerate(reduce_m): #вычитание минимумов по столбцам
        if not cnt:
            continue
        el[1:]-= min_d_j
    H+=sum(min_d_i)+sum(min_d_j)
    return reduce_m, H


In [256]:
def list_coord_zeros(reduce_m): #TODO переделать индексацию, надо брать по 0 строке и столбцу
    """
    Функция поиска нолей и их индексов в reduce_m
    
    reduce_m: редуцированная матрица
    return: list_coord_zeros - список кортежей с координатами нолей
    """       
    list_coord_zeros = []
    for cnt_i, i in enumerate(reduce_m): # from city
        for cnt_j, j in enumerate(i): # in city
            if j==0:
                list_coord_zeros.append((reduce_m[cnt_i][0], reduce_m[0][cnt_j])) # мог спутать i j
    return list_coord_zeros

In [257]:
def convert_indx(reduce_m, coord):
    """
    Вспомогательная функция конвертирует "мнимый" индекс матрицы в "настоящий"
    
    reduce_m: матрица для конвертации индексов
    coord: координаты с первоначальной матрицы
    return: row, col - координаты матрицы в номерации Python
    """
    col = list(reduce_m[0]).index(coord[1])
    row = list(reduce_m[:,0]).index(coord[0])
    return row, col

In [258]:
def search_branch(list_coord_zeros, reduce_m, list_branch=[]):
    """
    Функция поиска веток для отслеживания
    
    list_coord_zeros:список кортежей с координатами нолей в редуцированной матрице
    reduce_m: редуцированная матрица
    list_branch: пустой список, переменная на вырост если добавлять функционал ходить сразу по разным веткам
    return: list_branch - список координат (согласно первоначальной матрице) нулевых клеток с максимальной оценкой
    """
    sum_zero_cells = dict()
    for cnt, el in enumerate(list_coord_zeros): # Вычисление оценок нулевых клеток
        row, col = convert_indx(reduce_m, el)
        reduce_m[row][col] = inf # заменить ноль на inf
        sum_zero_cells[el] = reduce_m[row][1:].min()+reduce_m[:, col][1:].min() # Словарь нулевых клеток с максимальной оценкой   
        reduce_m[row][col] = 0
    for k, v in sum_zero_cells.items():
        if v==max(sum_zero_cells.values()):
            list_branch.append(k)
    return list_branch

In [259]:
def matrix_truncation(matrix, branch):
    """
    Функция отрезания строк и столбцов матрицы
    
    matrix: матрица которая усекается
    branch: координаты по которым усекается
    return: matrix - усеченная матрица
    """
    # TODO удалять не по брэнч индексу, а по нулевым, требует теста
    matrix = np.delete(matrix, list(matrix[0]).index(branch[1]), axis=1) # delete row
    matrix = np.delete(matrix, list(matrix[:,0]).index(branch[0]), axis=0) # delete col
    return matrix

In [260]:
def refresh_inf(matrix, inf=inf):
    """
    Функция обновляет в матрице inf
    
    matrix: матрица в которой обнавляют инф
    inf: "бесконечность" в матрице
    return: matrix - обновленная матрица
    """
    for i in range(len(matrix)):
        for j in range(len(matrix)):
            if matrix[i][j] > inf/2:
                matrix[i][j] = inf
    return matrix

In [261]:
def remove_looping(matrix):
    """
    Функция ищет строку и столбец не содержащие inf и ставит на их пересечение inf
    
    matrix: матрица в которой убираются возможные циклы
    return: matrix - обновленная матрица
    """
    matrix = refresh_inf(matrix)
    row = list(matrix.max(axis=1)) # максимумы по строкам
    row = row.index(min(row))
    
    col = list(matrix.max(axis=0)) # максимумы по столбцам
    col = col.index(min(col))
    
    matrix[row][col] = inf
    
    return matrix
            

In [262]:
def sorted_ready_path(ready_path, start):
    """
    Функция сортировки маршрута
    
    ready_path: список кортежей всех переходов коммивояжера
    start: пункт из которого стартуем
    return sorted_ready_p - отсортированный (правильный) список переходов коммивояжера
    """
    sorted_ready_p = []
    for x, y in ready_path:
        if x==start: # Стартовая точка
            sorted_ready_p.append((x,y))
            ready_path.remove((x,y))
            break
    while True:
        for x, y in ready_path:
            if sorted_ready_p[-1][1]==x:
                sorted_ready_p.append((x,y))
                ready_path.remove((x,y))
                break
        if len(ready_path)==0:
            break
    return sorted_ready_p

In [263]:
def drow_path(sorted_ready_path, data):
    """
    Функция отрисовки вывода
    
    sorted_ready_path: отсортированный (правильный) список переходов коммивояжера
    data: словарь с координатами
    return: s - строка вывода
    """
    matrix_len = distance_matrix(dict_coord=data)
    s = str(data[sorted_ready_path[0][0]])+' -> '
    lenght = 0
    for x, y in sorted_ready_path:
        lenght += matrix_len[int(x), int(y)]
        s += str(data[y]) + str([lenght]) + ' -> '
    s = s[:-4] + ' = ' + str([lenght])
    return s

In [271]:
data = {'Почтовое отделение':(0, 2),
        'Ул. Грибоедова, 104/25':(2, 5),
        'Ул. Бейкер стрит, 221б':(5, 2),
        'Ул. Большая Садовая, 302-бис':(6, 6),
        'Вечнозелёная Аллея, 742':(8, 3),
       }

# TODO сделать связь текстового и численного ключа

data = {1:(0, 2),
        2:(2, 5),
        3:(5, 2),
        4:(6, 6),
        5:(8, 3),
       }
inf = 10**5 # Обозначение бесконечности
# TODO изменить работу с inf, при больших значениях в матрице расстояний будет криво работать

H = 0 # Нижняя граница старт
start = 1 # Ключ стартовой точки


In [272]:
matrix = distance_matrix(dict_coord=data, inf=inf)
ready_path = []
while True:
    reduce_m = reduce_matrix(reduce_m=matrix, H=H) #-> reduce_m, H
    H = reduce_m[1]
    list_coord_z = list_coord_zeros(reduce_m[0]) # список кортежей с координатами нулей
    list_branch = search_branch(list_coord_zeros=list_coord_z,
                                reduce_m=reduce_m[0],
                                list_branch=[]
                               ) #список с координатами "максимальных нулей"
    ready_path.append(list_branch[0])
    # TODO надо list_branch для ветвления запользовать, пока ходит по одной ветке
    matrix = matrix_truncation(matrix=matrix, branch=list_branch[0]) # срезание матрицы по индексам
    matrix = remove_looping(matrix=matrix) # убирание возможного зацикливания
    if matrix.shape[0]==3:
        break
        
append_tmp = (matrix[1][0], matrix[0][2])
ready_path.append(append_tmp)
append_tmp = (matrix[2][0], matrix[0][1])
ready_path.append(append_tmp)
sorted_ready_p = sorted_ready_path(ready_path, start=start)
print(drow_path(sorted_ready_path=sorted_ready_p, data=data))

(0, 2) -> (2, 5)[3.605551275463989] -> (6, 6)[7.728656901081649] -> (8, 3)[11.334208176545639] -> (5, 2)[14.496485836714019] -> (0, 2)[19.49648583671402] = [19.49648583671402]
